In [1]:
import pandas as pd
import numpy as np

In [2]:
ss = pd.read_excel('Site Assessment/Northern Branch Phase II Debris Removal Ops.xlsx')
sa = pd.read_excel('Site Assessment/TT_North Branch_Site Assessment.xlsx', header=1)

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
#Rename SA columne headers to match SS
sa.rename({'Automobiles': 'Number of Vehicles'}, axis=1, inplace=True)


In [4]:
# ss.rename({'Site Assessment': 'Smart Sheets SA Completed'},axis=1, inplace=True)

In [5]:
#clean up APNS
ss['APN'] = ss['APN'].str.replace('-000', '').str.replace('-', '').str.extract(r'(\d{1,9}_*#*.*)')
sa['APN'] = sa['APN'].str.replace('-000', '').str.replace('-', '').str.extract(r'(\d{1,9}_*#*.*)')

In [6]:
#make everything lower case using a function
ss['APN'] = ss['APN'].apply(lambda x: x.lower())


In [7]:
sa['APN'] = sa['APN'].apply(lambda x: x.lower())

# test area to highlight differences and combine data frames

In [8]:
#fill NaNs with 0 and make an int and not a float
sa['Number of Vehicles'] = sa['Number of Vehicles'].fillna(0).astype(int)
sa['Number of Vehicles'].head()

0    0
1    0
2    1
3    1
4    3
Name: Number of Vehicles, dtype: int32

In [9]:
# has wprds in the number column made everything numeric and the fill nan with zeros
ss['Number of Vehicles'] = pd.to_numeric(ss.loc[:,'Number of Vehicles'], errors='coerce').fillna(0)
ss['Number of Vehicles'].astype(int)

0        1
1        7
2        1
3        1
4        5
        ..
2331    17
2332    34
2333     3
2334     2
2335    10
Name: Number of Vehicles, Length: 2336, dtype: int32

In [ ]:
sa.columns

In [10]:
# grab just columsn we need for practice
ss = ss[['APN', 'Site Assessment','Number of Vehicles']]
sa = sa[['APN', 'Date Completed','Number of Vehicles']]

In [11]:
# combine dataframes
dfall = pd.concat([ss.set_index('APN'), sa.set_index('APN')], axis='columns', keys=['Smart Sheets', 'Site Assessment'])
dfall.head()

Smart Sheets                    Site Assessment  \
          Site Assessment Number of Vehicles  Date Completed   
APN                                                            
061640013      2021-09-03                1.0      2021-09-03   
061020092      2021-04-23                7.0      2021-04-23   
071360009      2021-03-19                1.0      2021-03-19   
061020114      2021-01-11                1.0      2021-01-11   
061460008      2021-01-09                5.0      2021-01-09   

                              
          Number of Vehicles  
APN                           
061640013                1.0  
061020092                2.0  
071360009                0.0  
061020114                0.0  
061460008                1.0

In [12]:
# swap columsnto put them by eac other
df_final = dfall.swaplevel(axis='columns')[ss.columns[1:]]
df_final

Site Assessment Number of Vehicles                
              Smart Sheets       Smart Sheets Site Assessment
APN                                                          
061640013       2021-09-03                1.0             1.0
061020092       2021-04-23                7.0             2.0
071360009       2021-03-19                1.0             0.0
061020114       2021-01-11                1.0             0.0
061460008       2021-01-09                5.0             1.0
...                    ...                ...             ...
0200511800             NaT                NaN             3.0
062250026              NaT                NaN             0.0
0200802800             NaT                NaN             0.0
062290030              NaT                NaN             0.0
067300069              NaT                NaN             0.0

[2341 rows x 3 columns]

In [14]:
# i need someon to explain this to me

def highlight_diff(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Smart Sheets', axis='columns', level=-1)
    
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                      index=data.index, columns=data.columns)
df =  df_final.style.apply(highlight_diff, axis=None)
df

In [15]:
df.to_excel('Test34.xlsx')